## Weekly Inventory List

This script generates a formatted Excel file for doing weekly inventory checks in the MCL (currently Sunday afternoons)

NOTE that the API only provides data for items that are IN STOCK -- i.e. anything with "Checked Out" status is NOT included

NOTE that Weekly Inventory Files are stored on my computer at
/Users/krisjoseph/Library/Group Containers/UBF8T346G9.OneDriveSyncClientSuite/York University.noindex/York University/LIB-Media Creation Lab - Metrics and Tracking/Weekly Inventory/

In [1]:
import requests, json, datetime
from datetime import *
import pandas as pd

# Get today's date
datadate = datetime.today().strftime('%Y-%m-%d')
filepath="/Users/krisjoseph/Library/Group Containers/UBF8T346G9.OneDriveSyncClientSuite/York University.noindex/York University/LIB-Media Creation Lab - Metrics and Tracking/Weekly Inventory/"

# Get an API token for Springshare


In [2]:
## These values are specific to the Scott MCL instance of LibCal's Spaces and Equipment modules. 
## FYI, these IDs can be found in the LibCal admin web interface (you'll have to generate a clientSecret in the API module though)
locationID = 2632                                 # This is the system's location ID for the Scott MCL, which can be seen in the web-based admin for Libcal
clientID = 193                                    # Libapps client ID for the yorku account
clientSecret = 'b7ca8732e0786a851d971dc50d99f107' # Access password, generated using the admin interface for libcal (API module)

# URLs and data structures for API calls are all listed in the admin pages for the Libapps API module
url = 'https://yorku.libcal.com/1.1/oauth/token'
myRequestData = {'client_id': clientID,
        'client_secret': clientSecret,
        'grant_type': 'client_credentials'}

# send the request
call = requests.post(url, data = myRequestData)

# API authorization is returned in a JSON object, and we need to grab/store our access token, which
# is used to validate API calls for getting/setting data
authorizationData = call.json()
accessToken = authorizationData['access_token']

# Build and send query

In [3]:

#  API CALL https://yorku.libcal.com/1.1/equipment/items/2632  where 2632 is locationID
assetListURL = 'https://yorku.libcal.com/1.1/equipment/items/' + str(locationID)
pageIndex=0
isComplete = False
assetsPartial=[]
assetsComplete=[]

# NOTE for the following: the MAXIMUM record limit for the LibCal API is 100, so we need logic to check if there are still-more items
# to find after we run a query -- otherwise the inventory list gets cut off at 100 items

headers = {'Authorization':'Bearer '+accessToken}

while isComplete == False:

    assetData = {'pageSize': 100,
                 'visibility': 'admin_only',
                 'pageIndex': pageIndex}
    
    response = requests.get(assetListURL, headers=headers, params=assetData)
    assetsPartial = response.json()
    assetsComplete.extend(assetsPartial)
    if (len(assetsPartial)) < 100:
        isComplete = True
    else:
        pageIndex += 1
    
    



In [4]:
# Put data into a Pandas dataframe
inventoryList = pd.DataFrame.from_dict(assetsComplete, orient='columns')

inventoryList.drop(['termsAndConditions', 'description', 'image', 'instructions', 'value','replacement_cost', 
                    'formid', 'groupId', 'groupTermsAndConditions', 'locationTermsAndConditions',
                   'groupName', 'model'], axis=1)

# remove HTML tags from Damage Notes
inventoryList.damage_notes = inventoryList.damage_notes.replace(r'<[^<>]*>', '', regex=True)
inventoryList.damage_notes = inventoryList.damage_notes.replace(r'[\r\n|\r|\n|\t]', ' ', regex=True)

# Add column for DSA Notes
inventoryList["DSA_Notes"] = ""


In [5]:
# write output to Excel
writer = pd.ExcelWriter(filepath+datadate+" Loanable Inventory.xlsx", engine='xlsxwriter') 
#writer = pd.ExcelWriter(datadate+" Loanable Inventory.xlsx", engine='xlsxwriter') 

inventoryList.to_excel(writer, sheet_name=datadate, columns=['name','barcode','asset_number','serial_number','DSA_Notes','damage_notes'], 
                    index=False, freeze_panes=[1,2])


# worksheet is an instance of Excel sheet named after the inventory date - used for inserting the table
worksheet = writer.sheets[datadate]
# workbook is an instance of the whole book - used i.e. for cell format assignment 
workbook = writer.book

header_cell_format = workbook.add_format()
header_cell_format.set_align('center')
header_cell_format.set_align('vcenter')

# create list of dicts for header names 
#  (columns property accepts {'header': value} as header name)
col_names = [{'header': "Item Name"},
             {'header': "Barcode Number"},
             {'header': "Asset#"},
             {'header': "Serial#"},
             {'header': "Add Notes Here for Any Issues Found"},
             {'header': "Current Notes on Damage"}
            ]

worksheet.add_table(0, 0, inventoryList.shape[0], len(col_names), {'columns': col_names, 'style':'Table Style Medium 2'})

# skip the loop completly if AutoFit for header is not needed
for i, col in enumerate(col_names):
    # apply header_cell_format to cell on [row:0, column:i] and write text value from col_names in
    worksheet.write(0, i, col['header'], header_cell_format)

worksheet.autofit()
writer.close()
